# Featurelayer to Shapefile
This script is to import a feature layer from a live service to your S3 instance to be processed using the GeoProcessing script.

In [2]:
!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 41.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [geopandas]/2 [geopandas]


In [1]:
import geopandas as gpd
import requests
from shapely import geometry
import os
import json

# === CONFIGURATION ===
FEATURE_SERVICE_URL = "XXXXXXXX"  ### replace this with https://services-eu1.arcgis.com/jD89j6JX786MpRBU/ArcGIS/rest/services/NMPF_10_Defense_And_Security/FeatureServer
OUTPUT_FOLDER = "XXXXX"
OUTPUT_FILE = "XXXXXX" ################# S3

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
OUTPUT_PATH = os.path.join(OUTPUT_FOLDER, OUTPUT_FILE)

# === STEP 1: Try GeoJSON format ===
print("🔹 Querying feature service (trying GeoJSON)...")
params_geojson = {
    "where": "1=1",
    "outFields": "*",
    "f": "geojson"
}
response = requests.get(FEATURE_SERVICE_URL, params=params_geojson)

if response.status_code != 200 or not response.text.strip().startswith("{"):
    print("⚠️ GeoJSON request failed or invalid; trying Esri JSON instead...")
    params_esri = {
        "where": "1=1",
        "outFields": "*",
        "f": "json",
        "outSR": 4326
    }
    response = requests.get(FEATURE_SERVICE_URL, params=params_esri)
    data = response.json()

    if "features" not in data:
        print("❌ The service didn’t return a 'features' field — please check the URL.")
        print(json.dumps(data, indent=2))
        raise SystemExit()

    # Esri JSON → GeoDataFrame
    print("🔹 Converting from Esri JSON...")
    features = []
    for feat in data["features"]:
        geom = geometry.shape(feat["geometry"])
        attrs = feat.get("attributes", {})  # safer access
        features.append({**attrs, "geometry": geom})

    gdf = gpd.GeoDataFrame(features, crs="EPSG:4326")

else:
    # GeoJSON path
    print("🔹 Converting from GeoJSON...")
    data = response.json()
    gdf = gpd.GeoDataFrame.from_features(data["features"])

    # Set CRS safely
    if gdf.crs is None:
        gdf.set_crs(epsg=3857, inplace=True) ##### This may be 4326 if you're layer is on WGS84, check your ESRI WID.

# === SAVE SHAPEFILE ===
print(f"💾 Saving shapefile to {OUTPUT_PATH} ...")
gdf.to_file(OUTPUT_PATH)
print("✅ Shapefile exported successfully!")
print("CRS:", gdf.crs)

ModuleNotFoundError: No module named 'geopandas'

## Import to S3     

In [ ]:
import os

dir_output = os.path.abspath(pwd) #f'./data'
print(dir_output)

from upload_model import upload_model_to_s3_bucket

upload_model_to_s3_bucket(dir_output)